<a href="https://colab.research.google.com/github/JanetteFu/Message_Detect/blob/main/MH_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import pandas as pd
tweets = pd.read_csv("clean_tweet_hatespeech.csv")


tweets['label'] = tweets['label'].replace(1,4)
tweets['label'] = tweets['label'].replace(0,1)
tweets['label'] = tweets['label'].replace(4,0)
list(tweets.columns.values)

['Unnamed: 0', 'text', 'hashtags', 'emojis', 'label']

In [ ]:
tweets = tweets.sample(frac=1)
tweets.head()

,Unnamed: 0,text,hashtags,emojis,label
27216,29275,what s happing in # orlando is really shocking we must remember that behind the death toll was someone son or daughter,['#orlando'],[],1
5150,5534,be n # healthy every day do something that will inch you closer to a better tomorrow doug firebaugh,['#healthy'],[],1
30109,5643,can not wait to see u your admin hang for # treason # trump # make america great again,"['#treason', '#trump', '#makeamericagreatagain']",[],0
960,1037,like the spread of peanut butter on white bread # little wonders,['#littlewonders'],[],1
21549,23162,# cute,['#cute'],[],1


In [ ]:
label_counts = tweets.label.value_counts()
print(label_counts)

1    29720
0    2242 
Name: label, dtype: int64


In [ ]:
stop_words = pd.read_csv("stopwords2.csv")
stop_words = stop_words['0'].tolist()
stop_words = set(stop_words)


In [ ]:
import re, nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
#from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()



def normalizer(tweet):
  tokens = nltk.word_tokenize(tweet)
  filtered_result = list(filter(lambda l: l not in stop_words, tokens))
  lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
  return lemmas


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pd.set_option('display.max_colwidth', -1)
tweets['text'] = tweets.text.apply(str)
tweets['normalized_tweet'] = tweets.text.apply(normalizer)
tweets[['text', 'normalized_tweet']].head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,text,normalized_tweet
27216,what s happing in # orlando is really shocking we must remember that behind the death toll was someone son or daughter,"[s, happing, #, orlando, shocking, must, remember, behind, death, toll, someone, son, or, daughter]"
5150,be n # healthy every day do something that will inch you closer to a better tomorrow doug firebaugh,"[n, #, healthy, every, something, inch, closer, a, better, tomorrow, doug, firebaugh]"
30109,can not wait to see u your admin hang for # treason # trump # make america great again,"[wait, u, admin, hang, #, treason, #, trump, #, make, america, great, again]"
960,like the spread of peanut butter on white bread # little wonders,"[spread, peanut, butter, white, bread, #, little, wonder]"
21549,# cute,"[#, cute]"


In [ ]:
from nltk import ngrams
def ngrams(input_list):
  onegrams = input_list #try with unigrams
  bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
  trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
  return onegrams + bigrams + trigrams

tweets['grams'] = tweets.normalized_tweet.apply(ngrams)
tweets[['grams']].head()

,grams
27216,"[s, happing, #, orlando, shocking, must, remember, behind, death, toll, someone, son, or, daughter, s happing, happing #, # orlando, orlando shocking, shocking must, must remember, remember behind, behind death, death toll, toll someone, someone son, son or, or daughter, s happing #, happing # orlando, # orlando shocking, orlando shocking must, shocking must remember, must remember behind, remember behind death, behind death toll, death toll someone, toll someone son, someone son or, son or daughter]"
5150,"[n, #, healthy, every, something, inch, closer, a, better, tomorrow, doug, firebaugh, n #, # healthy, healthy every, every something, something inch, inch closer, closer a, a better, better tomorrow, tomorrow doug, doug firebaugh, n # healthy, # healthy every, healthy every something, every something inch, something inch closer, inch closer a, closer a better, a better tomorrow, better tomorrow doug, tomorrow doug firebaugh]"
30109,"[wait, u, admin, hang, #, treason, #, trump, #, make, america, great, again, wait u, u admin, admin hang, hang #, # treason, treason #, # trump, trump #, # make, make america, america great, great again, wait u admin, u admin hang, admin hang #, hang # treason, # treason #, treason # trump, # trump #, trump # make, # make america, make america great, america great again]"
960,"[spread, peanut, butter, white, bread, #, little, wonder, spread peanut, peanut butter, butter white, white bread, bread #, # little, little wonder, spread peanut butter, peanut butter white, butter white bread, white bread #, bread # little, # little wonder]"
21549,"[#, cute, # cute]"


In [ ]:
import collections
def count_words(input):
  count = collections.Counter()
  for row in input:
    for word in row:
      count[word] += 1
  return count

In [ ]:
tweets[(tweets.label == 0)][['grams']].apply(count_words)['grams'].most_common(50)


[('#', 4810),
 ('a', 679),
 ('s', 353),
 ('i', 292),
 ('trump', 280),
 ('white', 213),
 ('black', 190),
 ('# trump', 165),
 ('woman', 137),
 ('w', 134),
 ('re', 132),
 ('u', 130),
 ('by', 119),
 ('racist', 119),
 ('new', 110),
 ('people', 107),
 ('m', 105),
 ('racism', 99),
 ('politics', 97),
 ('why', 96),
 ('obama', 96),
 ('# politics', 95),
 ('who', 94),
 ('liberal', 93),
 ('allah', 92),
 ('soil', 92),
 ('# allah', 92),
 ('allah soil', 92),
 ('# allah soil', 92),
 ('hate', 89),
 ('a #', 88),
 ('his', 87),
 ('right', 86),
 ('# black', 84),
 ('tweet', 83),
 ('# liberal', 82),
 ('lib', 80),
 ('# lib', 79),
 ('feminism', 78),
 ('w #', 77),
 ('might', 77),
 ('tard', 77),
 ('lib tard', 77),
 ('# lib tard', 77),
 ('sj', 76),
 ('# sj', 76),
 ('sj w', 76),
 ('# sj w', 76),
 ('t', 75),
 ('liberal #', 75)]

In [ ]:
tweets[(tweets.label == 1)][['grams']].apply(count_words)['grams'].most_common(50)


[('#', 71817),
 ('i', 7803),
 ('a', 6973),
 ('s', 3996),
 ('love', 3537),
 ('# #', 2145),
 ('# love', 2051),
 ('happy', 1853),
 ('life', 1814),
 ('u', 1804),
 ('love #', 1482),
 ('father', 1331),
 ('# love #', 1221),
 ('m', 1211),
 ('new', 1197),
 ('i m', 999),
 ('positive', 998),
 ('thankful', 960),
 ('# positive', 934),
 ('smile', 902),
 ('a #', 885),
 ('friend', 867),
 ('way', 861),
 ('our', 857),
 ('make', 854),
 ('girl', 840),
 ('people', 826),
 ('by', 775),
 ('life #', 756),
 ('fun', 730),
 ('family', 707),
 ('want', 699),
 ('best', 692),
 ('insta', 681),
 ('weekend', 678),
 ('# smile', 677),
 ('take', 673),
 ('orlando', 669),
 ('friday', 667),
 ('summer', 665),
 ('week', 654),
 ('healthy', 651),
 ('# i', 642),
 ('i #', 642),
 ('re', 642),
 ('# insta', 639),
 ('s #', 627),
 ('music', 618),
 ('# healthy', 614),
 ('who', 607)]

In [ ]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
#count_vectorizer = CountVectorizer(ngram_range=(1,2))

import pickle
pkl_filename = 'count_vectorizer.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(count_vectorizer, file)


# Load from file
#with open('SVM_Hatespeech.pkl', 'rb') as file:
    #pickle_model = pickle.load(file)

In [ ]:
vectorized_data = count_vectorizer.fit_transform(tweets.text)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))


In [ ]:
targets = tweets.label

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(class_weight='balanced')
logisticRegr.fit(data_train, targets_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#from sklearn.metrics import classification_report
predictions = logisticRegr.predict(data_test)
score = logisticRegr.score(data_test, targets_test)
print(score)

0.9590926867422761


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(targets_test, predictions))  

              precision    recall  f1-score   support

           0       0.75      0.65      0.70       924
           1       0.97      0.98      0.98     11861

    accuracy                           0.96     12785
   macro avg       0.86      0.81      0.84     12785
weighted avg       0.96      0.96      0.96     12785



In [ ]:

#import pickle

pkl_filename = 'logisticRegrHateSpeech.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(logisticRegr, file)

# Load from file
with open('logisticRegrHateSpeech.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
# Calculate the accuracy score and predict target values
score = pickle_model.score(data_test, targets_test)
print('Test score: {0:.2f} %'.format(100 * score))
Ypredict = pickle_model.predict(data_test)


Test score: 95.91 %


In [ ]:
mislabeled_logreg = []
for input, prediction, label in zip(tweets.text, predictions, targets_test):
  if prediction != label:
    #print(input, 'has been classified as ', prediction, 'and should be ', label) 
    mislabeled_logreg.append(input)
len(mislabeled_logreg)

119686

In [ ]:

from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=0.1, probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)
clf.score(data_test, targets_test)





0.763046875

In [ ]:
predictions = clf.predict(data_test)
print(classification_report(targets_test, predictions))  

              precision    recall  f1-score   support

           0       0.78      0.73      0.75      6390
           1       0.75      0.80      0.77      6410

    accuracy                           0.76     12800
   macro avg       0.76      0.76      0.76     12800
weighted avg       0.76      0.76      0.76     12800



In [ ]:
'''
pkl_filename = 'SVM_Hatespeech.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)
'''

# Load from file
with open('SVM_Hatespeech.pkl', 'rb') as file:
    pickle_model = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(data_test, targets_test)
print('Test score: {0:.2f} %'.format(100 * score))
Ypredict = pickle_model.predict(data_test)

Test score: 86.73 %


In [ ]:
mislabeled_svm = []
for input, prediction, label in zip(tweets.text, predictions, targets_test):
  if prediction != label:
    #print(input, 'has been classified as ', prediction, 'and should be ', label) 
    mislabeled_svm.append(input)
len(mislabeled_svm)

3033

In [ ]:
print(len(mislabeled_logreg))
print(len(mislabeled_svm))
print(len(set(mislabeled_logreg).intersection(mislabeled_svm)))
print(len(set(mislabeled_logreg).difference(mislabeled_svm)))
print(len(set(mislabeled_svm).difference(mislabeled_logreg)))



2983
3033
2682
272
324
